# Monitoring Large Language Models (LLMs) with LangKit & Hugging Face

[LangKit](https://github.com/whylabs/langkit) - open-source набор инструментов для мониторинга больших языковых моделей (LLM). Извлекает метрики из промптов и ответов модели. В числе метрик реализованы:

- [Text Quality](https://github.com/whylabs/langkit/blob/main/langkit/docs/features/quality.md)
- [Text Relevance](https://github.com/whylabs/langkit/blob/main/langkit/docs/features/relevance.md)
- [Security and Privacy](https://github.com/whylabs/langkit/blob/main/langkit/docs/features/security.md)
- [Sentiment and Toxicity](https://github.com/whylabs/langkit/blob/main/langkit/docs/features/sentiment.md)



## Setup

- [Free WhyLabs Account](https://whylabs.ai/free)




In [1]:
%%capture
!pip install transformers 'langkit[all]' -U

In [2]:
import os
import datetime

import pandas as pd
import whylogs as why
import langkit.toxicity

from whylogs.api.writer.whylabs import WhyLabsWriter


from langkit import llm_metrics
from langkit.whylogs.samples import load_chats, show_first_chat

from transformers import GPT2LMHeadModel, GPT2Tokenizer

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/403 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [3]:
pd.set_option("display.max_columns", None)

## 👋 Hello, World! Пример

Тестовый запуск на парах промпт/ответ



In [4]:
schema = llm_metrics.init()

In [5]:
chats = load_chats()
print(f"There are {len(chats)} records in this toy example data, here's the first one:")
show_first_chat(chats)

results = why.log(chats, name="langkit-sample-chats-all", schema=schema)

There are 50 records in this toy example data, here's the first one:
prompt: Hello, response: World!

⚠️ No session found. Call whylogs.init() to initialize a session and authenticate. See https://docs.whylabs.ai/docs/whylabs-whylogs-init for more information.


In [6]:
chats

,prompt,response
0,"Hello,",World!
1,"Hello, World!",Hello! How can I assist you today?
2,Aproximately how many atoms are in the known u...,There are approximately 10^80 atoms in the obs...
3,What is the speed of light in m/s? Can you out...,The speed of light in a vacuum is approximatel...
4,How many digits are in a Discover credit card ...,A Discover credit card number has 16 digits. T...
5,Can you give me an example of a string that ma...,"Here is an example of a fake, randomly generat..."
6,Can you give me three more examples. Also can ...,"fakeDiscoverNumbers: 6011-1234-5678-9123, 6011..."
7,Can you give me two more examples of fake cred...,"fakeDiscoverNumbers: 6011987654321098, 6011555..."
8,l/KSADJF SDFLJ HO GObble glid foisjka alskjfda...,5.
9,Can you give me 2 fake phone numbers?,"Here are 2 fake phone numbers: (555) 123-4567,..."


In [7]:
profview = results.view()
profview.to_pandas()

,cardinality/est,cardinality/lower_1,cardinality/upper_1,counts/inf,counts/n,counts/nan,counts/null,distribution/max,distribution/mean,distribution/median,distribution/min,distribution/n,distribution/q_01,distribution/q_05,distribution/q_10,distribution/q_25,distribution/q_75,distribution/q_90,distribution/q_95,distribution/q_99,distribution/stddev,type,types/boolean,types/fractional,types/integral,types/object,types/string,types/tensor,ints/max,ints/min,frequent_items/frequent_strings
column,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
prompt,47.000005,47.0,47.002352,0,50,0,0,NaN,0.000000,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,SummaryType.COLUMN,0,0,0,0,50,0,NaN,NaN,NaN
prompt.aggregate_reading_level,17.000001,17.0,17.000849,0,50,0,0,28.000000,7.580000,8.000000,0.000000,50,0.000000,0.000000,2.000000,5.000000,9.000000,13.000000,14.000000,28.000000,4.849111,SummaryType.COLUMN,0,50,0,0,0,0,NaN,NaN,NaN
prompt.automated_readability_index,38.000003,38.0,38.001901,0,50,0,0,31.200000,7.998000,7.400000,-3.000000,50,-3.000000,2.100000,3.200000,5.200000,10.400000,13.100000,15.800000,31.200000,5.198449,SummaryType.COLUMN,0,50,0,0,0,0,NaN,NaN,NaN
prompt.character_count,40.000004,40.0,40.002001,0,50,0,0,282.000000,81.280000,59.000000,6.000000,50,6.000000,12.000000,33.000000,46.000000,103.000000,167.000000,218.000000,282.000000,60.255253,SummaryType.COLUMN,0,0,50,0,0,0,282.0,6.0,NaN
prompt.difficult_words,11.000000,11.0,11.000549,0,50,0,0,24.000000,3.980000,4.000000,0.000000,50,0.000000,0.000000,1.000000,2.000000,5.000000,6.000000,8.000000,24.000000,3.565652,SummaryType.COLUMN,0,0,50,0,0,0,24.0,0.0,NaN
prompt.flesch_reading_ease,43.000004,43.0,43.002151,0,50,0,0,118.180000,63.868600,67.760000,-35.800000,50,-35.800000,15.640000,36.620000,55.240000,77.230000,91.480000,92.800000,118.180000,25.373953,SummaryType.COLUMN,0,50,0,0,0,0,NaN,NaN,NaN
prompt.has_patterns,2.000000,2.0,2.000100,0,50,0,46,NaN,0.000000,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,SummaryType.COLUMN,0,0,0,0,4,0,NaN,NaN,"[FrequentItem(value='SSN', est=2, upper=2, low..."
prompt.jailbreak_similarity,46.000005,46.0,46.002302,0,50,0,0,1.000000,0.307416,0.277097,0.107413,50,0.107413,0.121107,0.148813,0.211474,0.311718,0.452144,1.000000,1.000000,0.195855,SummaryType.COLUMN,0,50,0,0,0,0,NaN,NaN,NaN
prompt.letter_count,37.000003,37.0,37.001851,0,50,0,0,274.000000,78.660000,58.000000,5.000000,50,5.000000,10.000000,29.000000,45.000000,101.000000,164.000000,213.000000,274.000000,59.047407,SummaryType.COLUMN,0,0,50,0,0,0,274.0,5.0,NaN


##🤗 LangKit & HuggingFace model

Импортируем GPT2

In [8]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Функция для обработки промпта и генерации ответа

In [9]:
def gpt_model(prompt):
  input_ids = tokenizer.encode(prompt, return_tensors='pt')

  output = model.generate(
      input_ids,
      max_length=100,
      temperature=0.8,
      do_sample=True,
      pad_token_id=tokenizer.eos_token_id,
  )

  response = tokenizer.decode(output[0], skip_special_tokens=True)

  prompt_and_response = {
      "prompt": prompt,
      "response": response
  }

  return prompt_and_response

In [10]:
prompt_and_response = gpt_model("Tell me a story about a cute dog")
print(prompt_and_response)

{'prompt': 'Tell me a story about a cute dog', 'response': 'Tell me a story about a cute dog that goes out to eat and then comes back and comes back again, but is not eaten by the dog for that long. He goes to go out with friends. It isn\'t until the dogs come back that I see the "puppy."\n\nA very rare occurrence that is a common occurrence in American dogs. It occurs when one dog is seen playing with a puppy/dog. That is when the puppy or dog is in the puppy/dog'}


### Create & Inspect Language Metrics with LangKit

LangKit provides a toolkit of metrics for LLM applications, lets initialize them and create a profile of the data that can be viewed in WhyLabs for quick analysis.

In [11]:
profile = why.log(prompt_and_response, schema=schema).profile()

In [12]:
profview = profile.view()
profview.to_pandas()

,cardinality/est,cardinality/lower_1,cardinality/upper_1,counts/inf,counts/n,counts/nan,counts/null,distribution/max,distribution/mean,distribution/median,distribution/min,distribution/n,distribution/q_01,distribution/q_05,distribution/q_10,distribution/q_25,distribution/q_75,distribution/q_90,distribution/q_95,distribution/q_99,distribution/stddev,type,types/boolean,types/fractional,types/integral,types/object,types/string,types/tensor,ints/max,ints/min,frequent_items/frequent_strings
column,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
prompt,1.0,1.0,1.00005,0,1,0,0,NaN,0.000000,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,SummaryType.COLUMN,0,0,0,0,1,0,NaN,NaN,NaN
prompt.aggregate_reading_level,1.0,1.0,1.00005,0,1,0,0,1.000000,1.000000,1.000000,1.000000,1,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,SummaryType.COLUMN,0,1,0,0,0,0,NaN,NaN,NaN
prompt.automated_readability_index,1.0,1.0,1.00005,0,1,0,0,-2.800000,-2.800000,-2.800000,-2.800000,1,-2.800000,-2.800000,-2.800000,-2.800000,-2.800000,-2.800000,-2.800000,-2.800000,0.0,SummaryType.COLUMN,0,1,0,0,0,0,NaN,NaN,NaN
prompt.character_count,1.0,1.0,1.00005,0,1,0,0,25.000000,25.000000,25.000000,25.000000,1,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,0.0,SummaryType.COLUMN,0,0,1,0,0,0,25.0,25.0,NaN
prompt.difficult_words,1.0,1.0,1.00005,0,1,0,0,0.000000,0.000000,0.000000,0.000000,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,SummaryType.COLUMN,0,0,1,0,0,0,0.0,0.0,NaN
prompt.flesch_reading_ease,1.0,1.0,1.00005,0,1,0,0,105.660000,105.660000,105.660000,105.660000,1,105.660000,105.660000,105.660000,105.660000,105.660000,105.660000,105.660000,105.660000,0.0,SummaryType.COLUMN,0,1,0,0,0,0,NaN,NaN,NaN
prompt.has_patterns,0.0,0.0,0.00000,0,1,0,1,NaN,0.000000,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,SummaryType.COLUMN,0,0,0,0,0,0,NaN,NaN,[]
prompt.jailbreak_similarity,1.0,1.0,1.00005,0,1,0,0,0.143321,0.143321,0.143321,0.143321,1,0.143321,0.143321,0.143321,0.143321,0.143321,0.143321,0.143321,0.143321,0.0,SummaryType.COLUMN,0,1,0,0,0,0,NaN,NaN,NaN
prompt.letter_count,1.0,1.0,1.00005,0,1,0,0,25.000000,25.000000,25.000000,25.000000,1,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,0.0,SummaryType.COLUMN,0,0,1,0,0,0,25.0,25.0,NaN


### Несколько промптов

In [13]:
prompts = ["What is AI?",
           "Tell me a joke.",
           "Who won the world series in 2021?"]

In [14]:
for num, prompt in enumerate(prompts):

  prompt_and_response = gpt_model(prompt)

  if num == 0:
    profile = why.log(prompt_and_response, schema=schema).profile()
  profile.track(prompt_and_response)

In [15]:
profview = profile.view()
profview.to_pandas()

,cardinality/est,cardinality/lower_1,cardinality/upper_1,counts/inf,counts/n,counts/nan,counts/null,distribution/max,distribution/mean,distribution/median,distribution/min,distribution/n,distribution/q_01,distribution/q_05,distribution/q_10,distribution/q_25,distribution/q_75,distribution/q_90,distribution/q_95,distribution/q_99,distribution/stddev,type,types/boolean,types/fractional,types/integral,types/object,types/string,types/tensor,ints/max,ints/min,frequent_items/frequent_strings
column,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
prompt,3.0,3.0,3.00015,0,4,0,0,NaN,0.000000,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,SummaryType.COLUMN,0,0,0,0,4,0,NaN,NaN,NaN
prompt.aggregate_reading_level,2.0,2.0,2.00010,0,4,0,0,1.000000,0.500000,1.000000,0.000000,4,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.577350,SummaryType.COLUMN,0,4,0,0,0,0,NaN,NaN,NaN
prompt.automated_readability_index,3.0,3.0,3.00015,0,4,0,0,0.300000,-4.225000,-5.400000,-5.900000,4,-5.900000,-5.900000,-5.900000,-5.900000,0.300000,0.300000,0.300000,0.300000,3.025861,SummaryType.COLUMN,0,4,0,0,0,0,NaN,NaN,NaN
prompt.character_count,3.0,3.0,3.00015,0,4,0,0,27.000000,14.250000,12.000000,9.000000,4,9.000000,9.000000,9.000000,9.000000,27.000000,27.000000,27.000000,27.000000,8.616844,SummaryType.COLUMN,0,0,4,0,0,0,27.0,9.0,NaN
prompt.difficult_words,2.0,2.0,2.00010,0,4,0,0,1.000000,0.250000,0.000000,0.000000,4,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.500000,SummaryType.COLUMN,0,0,4,0,0,0,1.0,0.0,NaN
prompt.flesch_reading_ease,3.0,3.0,3.00015,0,4,0,0,119.190000,115.807500,119.190000,106.670000,4,106.670000,106.670000,106.670000,118.180000,119.190000,119.190000,119.190000,119.190000,6.110245,SummaryType.COLUMN,0,4,0,0,0,0,NaN,NaN,NaN
prompt.has_patterns,0.0,0.0,0.00000,0,4,0,4,NaN,0.000000,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,SummaryType.COLUMN,0,0,0,0,0,0,NaN,NaN,[]
prompt.jailbreak_similarity,3.0,3.0,3.00015,0,4,0,0,0.521414,0.339021,0.521414,0.068118,4,0.068118,0.068118,0.068118,0.245137,0.521414,0.521414,0.521414,0.521414,0.222664,SummaryType.COLUMN,0,4,0,0,0,0,NaN,NaN,NaN
prompt.letter_count,3.0,3.0,3.00015,0,4,0,0,26.000000,13.250000,11.000000,8.000000,4,8.000000,8.000000,8.000000,8.000000,26.000000,26.000000,26.000000,26.000000,8.616844,SummaryType.COLUMN,0,0,4,0,0,0,26.0,8.0,NaN


## Интеграция с WhyLabs

Для использования LangKit с WhyLabs, нам понадобятся :

- API-токен
- Идентификатор организации
- Идентификатор набора данных (или идентификатор модели)

Перейдите на [https://whylabs.ai/free](https://whylabs.ai/free) и создайте бесплатную учетную запись.

1. Создайте новый проект и запишите его идентификатор (если это модельный проект, он будет выглядеть как «model-хххх»).
2. Создайте токен API на вкладке «Access Tokens».
3. Скопируйте идентификатор своей организации с той же вкладки «Access Tokens».



In [16]:
os.environ["WHYLABS_DEFAULT_ORG_ID"] = "org-qJ6k9T" # ORG-ID is case sensistive
os.environ["WHYLABS_API_KEY"] = '' #API Token
os.environ["WHYLABS_DEFAULT_DATASET_ID"] = 'model-2' #can also be provided as dataset_id param in WhyLabsWriter constructor

In [17]:
schema = llm_metrics.init()

In [18]:
telemetry_agent = WhyLabsWriter()
profile = why.log(prompt_and_response, schema=schema)
telemetry_agent.write(profile.view())

⚠️ Initializing default session because no session was found.
Initializing session with config /root/.config/whylogs/config.ini

✅ Using session type: LOCAL. Profiles won't be uploaded or written anywhere automatically.


(True, 'log-w4QQwrwHV2l7Rh2f')

Варианты использования собранных данных:
- Обеспечение безопасности LLM
- Prompt engineering эксперименты
- Изменение производительности модели на отдельных промптах
- Shadow Deployment
- Мониторинг дрифта данных



In [19]:
prompt_lists = [
    ["How can I create a new account?", "Great job to the team", "Fantastic product, had a good experience"],
    ["This product made me angry, can I return it? Give a phone number to call 800-987-6543", "You dumb and smell bad", "I hated the experience, and I was over charged"],
    ["This seems amazing, could you share the pricing?", "Incredible site, could we setup a call?", "Hello! Can you kindly guide me through the documentation?"],
    ["This looks impressive, could you provide some information on the cost?", "Stunning platform, can we arrange a chat?", "Hello there! Could you assist me with the documentation?"],
    ["This looks remarkable, could you tell me the price range?", "Fantastic webpage, is it possible to organize a call?", "Greetings! Can you help me with the relevant documents?"],
    ["This is great, Ilove it, could you inform me about the charges?", "love the interface, can we have a teleconference?", "Hello! Can I take a look at the user manuals?"],
    ["This seems fantastic, how much does it cost?", "Excellent website, can we setup a call?", "Hello! Could you help me find the resource documents?"]
]


In [20]:
telemetry_agent = WhyLabsWriter()
all_prompts_and_responses = []  # This list will store all the prompts and responses.

In [21]:
for i, day in enumerate(prompt_lists):
  telemetry_agent = WhyLabsWriter()
  # walking backwards. Each dataset has to map to a date to show up as a different batch in WhyLabs
  dt = datetime.datetime.now(tz=datetime.timezone.utc) - datetime.timedelta(days=i)
  for prompt in day:
    prompt_and_response = gpt_model(prompt)
    profile = why.log(prompt_and_response, schema=schema)

     # Save the prompt and its response in the list.
    all_prompts_and_responses.append(prompt_and_response)

    # set the dataset timestamp for the profile
    profile.set_dataset_timestamp(dt)
    telemetry_agent.write(profile.view())

In [22]:
all_prompts_and_responses

[{'prompt': 'How can I create a new account?',
  'response': 'How can I create a new account?\n\nCreate a new account with an existing username and password.\n\nUse an existing account with a valid email address to create a new account.\n\nIf you do not have an email address, you can create an email account using our email address service. You can also create a new account on our social media accounts.\n\nWhat are the steps to create a new account?\n\nYou need to create a new account on our social media page'},
 {'prompt': 'Great job to the team',
  'response': 'Great job to the team, good job to the manager and good job to the board and the chair and good job to everyone, that\'s my job.\n\n"I know how hard it is to work every day. Everyone\'s trying. We have a great team here."\n\nBruins coach Mike Babcock said he wanted to make a statement after Sunday\'s game but there wasn\'t enough action. He said the team was being "firmly held back" in trying to work'},
 {'prompt': 'Fantastic p

### Валидация



In [23]:
print("downloading models and initialized metrics...")
text_metrics_schema = llm_metrics.init()

downloading models and initialized metrics...


In [24]:
def is_not_toxic(prompt_message):
    profile = why.log({"prompt": prompt_message}, schema=text_metrics_schema).profile().view()

    profile_dict = profile.get_column('prompt.toxicity').to_summary_dict()
    tox_max = profile_dict['distribution/max']

    print(f'Toxic score: {tox_max}')

    if tox_max > 0.5:
      return False
    return True

In [25]:
is_not_toxic("Do you like fruit?")

Toxic score: 0.0014600753784179688


True

In [26]:
is_not_toxic("You dumb and smell bad")

Toxic score: 0.9606605768203735


False

In [27]:
user_prompt = "Do you like fruit?"

if is_not_toxic(user_prompt):
  prompt_and_response = gpt_model(user_prompt)
  print(prompt_and_response['response'])

else:
  print("As a large language model...")

Toxic score: 0.0014600753784179688
Do you like fruit? No, not at all. If you want to eat a lot of fruit, you can try these delicious fruit juices, which are made from all types of fruits like cucumbers, apricots, cherries, mangoes and even mangoes. The fruits are pureed which help to promote health and reduce the side effects of drinking sugary drinks.

3. Fruit Smoothies

There are many fruits that are available from all over the world. These are


In [28]:
user_prompt = "You dumb and smell bad"

if is_not_toxic(user_prompt):
  prompt_and_response = gpt_model(user_prompt)
  print(prompt_and_response['response'])

else:
  print("As a large language model...")

Toxic score: 0.9606605768203735
As a large language model...
